# 分类

> 使用带有结构化输出的聊天模型将文本分类到不同的类别或标签中。

## 将文本分类到标签中

“打标签”（Tagging）是指使用以下类型的类别对文档进行标注：

- **情感**（正面、负面、中性等）  
- **语言**（中文、英文、法语等）  
- **风格**（正式、非正式等）  
- **涉及的主题**  
- **政治倾向**


![示例图片](../assets/imgs/tagging.png)  

### 概述  
**打标签（Tagging）** 包含几个组成部分：

- **函数（function）**：和信息提取类似，打标签也使用函数来指定模型应该如何对文档进行标注  
- **模式（schema）** ：定义了我们希望如何对文档进行打标签  

---

### 快速开始  
下面我们来看一个非常简单的例子，展示如何在 LangChain 中使用 OpenAI 的工具调用功能来进行**文本打标签**。

我们会使用 OpenAI 模型支持的 `with_structured_output` 方法。

（注：这个方法可以让模型输出结构化的数据格式，比如 JSON，非常适合用来做分类、打标签等任务。）

In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    _ = load_dotenv()
except ImportError:
    pass

if not os.environ.get("DASHSCOPE_API_KEY"):
  os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_community.chat_models.tongyi import ChatTongyi

llm = ChatTongyi(
    streaming=True,
    name="qwen-turbo"
)

我们可以在模式（schema）中指定一个 Pydantic 模型，包含一些属性及其预期的数据类型。

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# 从以下段落中提取所需信息。
# 仅提取“分类”函数中提到的属性。
# 段落：

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

class Classification(BaseModel):
    # 文本的情感
    sentiment: str = Field(description="The sentiment of the text") 
    # 文字的攻击性程度（1 到 10 分
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    # 文本所用的语言
    language: str = Field(description="The language the text is written in")

# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [44]:
input_content = "哇，又迟到了，真是太棒了。"
prompt = tagging_prompt.invoke({"input": input_content})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='negative', aggressiveness=3, language='zh')

如果我们希望得到字典格式的输出，只需调用 `.model_dump()` 方法即可。

In [47]:
inp = "这个点子太‘绝’了。"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'positive', 'aggressiveness': 5, 'language': 'Chinese'}

In [50]:
if hasattr(response, "model_dump"):
    print(response.model_dump())
else:
    print(response)

{'sentiment': 'positive', 'aggressiveness': 5, 'language': 'Chinese'}


正如我们在示例中看到的那样，模型能正确理解我们的需求。

输出的结果可能会有所不同，例如，情感标签可能是不同语言的（如英文的 `"positive"`、中文的 `"积极"` 等）。

我们将在下面学习如何控制这些输出结果。

### 更精细的控制  
通过仔细定义模式（schema），我们可以更好地控制模型的输出结果。

具体来说，我们可以在模型中定义以下内容：

- 每个属性的**可选值**
- 属性的**描述信息**，确保模型准确理解每个字段的含义
- 必须返回的**必填属性**

下面我们重新定义我们的 Pydantic 模型，使用枚举（enum）来实现上述各个方面的控制：

In [52]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["开心", "中性", "悲伤"])
    aggressiveness: int = Field(
        ...,
        description="描述言论的攻击性，数字越高，攻击性越强",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["中文", "英文", "日本", "法国", "繁体中文"]
    )

In [76]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

chain = llm.with_structured_output(
    Classification
)

现在，回答将会以我们期望的方式被限制（规范）！

In [80]:
inp = "真是太棒了，连点外卖都能送错，客服还不接电话，简直五星好评！"
prompt = tagging_prompt.invoke({"input": inp})
response = chain.invoke(prompt)

if hasattr(response, "model_dump"):
    print(response.model_dump())
else:
    print(response)

{'sentiment': '开心', 'aggressiveness': 3, 'language': '中文'}
